# Installing packages

In [ ]:
!pip install transformers
!pip install -q datasets seqeval
!pip install pyyaml==5.1
!pip install torch==1.10.0+cu111 torchvision==0.11.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!python -m pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html

# Importing libraries

In [2]:
from ast import arg
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer,AutoTokenizer
from datasets import load_metric,load_from_disk
import os
import numpy as np
import torch
import warnings
import argparse

warnings.filterwarnings('ignore')

In [3]:
import os
os.chdir('/content/drive/MyDrive/unstop_competition/')

In [4]:
!mkdir model

mkdir: cannot create directory ‘model’: File exists


In [5]:
# Defining parameters for training
EPOCHS = 20
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
LEARNING_RATE = 5e-5
LR_SCHEDULER_TYPE = "linear"
WARMUP_RATIO = 0
output_dir = "model"
data_dir = "data/preprocessed_data_train"

In [6]:
# Loading train dataset
train_dataset = load_from_disk(f'{data_dir}')

In [7]:
labels = train_dataset.features["labels"].feature.names
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

# Loading microsft layoutlm2 .It has restrictive license which stops it for commercial usage
model = AutoModelForTokenClassification.from_pretrained('microsoft/layoutlmv2-base-uncased',
                                                                    num_labels=len(label2id))
tokenizer = AutoTokenizer.from_pretrained('microsoft/layoutlmv2-base-uncased')


Some weights of the model checkpoint at microsoft/layoutlmv2-base-uncased were not used when initializing LayoutLMv2ForTokenClassification: ['layoutlmv2.visual.backbone.bottom_up.res3.3.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.17.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.1.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.5.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.13.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.6.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res2.2.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res3.2.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res3.2.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res3.2.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.3.conv1.norm.num_batches_tracked',

In [8]:
model.config.id2label = id2label
model.config.label2id = label2id
# Metrics
metric = load_metric("seqeval")
return_entity_level_metrics = True

In [9]:
def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax(predictions, axis=2)
  # Remove ignored index (special tokens)
  true_predictions = [[id2label[p] for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)  ]
  true_labels = [[id2label[l] for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)]

  results = metric.compute(predictions=true_predictions, references=true_labels)
  if return_entity_level_metrics:
      # Unpack nested dictionaries
      final_results = {}
      for key, value in results.items():
          if isinstance(value, dict):
              for n, v in value.items():
                  final_results[f"{key}_{n}"] = v
          else:
              final_results[key] = value
      return final_results
  else:
      return {
          "precision": results["overall_precision"],
          "recall": results["overall_recall"],
          "f1": results["overall_f1"],
          "accuracy": results["overall_accuracy"],
      }

In [10]:
args = TrainingArguments(
        output_dir=output_dir, # name of directory to store the checkpoints
        evaluation_strategy = "epoch",
        logging_strategy = "epoch",
        num_train_epochs=EPOCHS,
        learning_rate = LEARNING_RATE,
        per_device_train_batch_size=TRAIN_BATCH_SIZE,
        per_device_eval_batch_size=VALID_BATCH_SIZE,
        metric_for_best_model = "overall_f1",
        lr_scheduler_type = LR_SCHEDULER_TYPE,  #constant/linear/cosine/cosine_with_restarts/polynomial/constant_with_warmup
        warmup_ratio = WARMUP_RATIO, # optional, defaults to 0.0
        # max_steps=EPOCHS*len(train_dataloader),
        # fp16=True, # we use mixed precision (less memory consumption)
        save_total_limit = 1,
        load_best_model_at_end=True,
        save_strategy = "epoch"
    )

In [11]:
# Training the model
trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=train_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
trainer.train()

You're using a LayoutLMv2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bill To Address Precision,Bill To Address Recall,Bill To Address F1,Bill To Address Number,Bill To Name Precision,Bill To Name Recall,Bill To Name F1,Bill To Name Number,Due Date Precision,Due Date Recall,Due Date F1,Due Date Number,Invoice Date Precision,Invoice Date Recall,Invoice Date F1,Invoice Date Number,Invoice Number Precision,Invoice Number Recall,Invoice Number F1,Invoice Number Number,Payment Info Precision,Payment Info Recall,Payment Info F1,Payment Info Number,Ship To Address Precision,Ship To Address Recall,Ship To Address F1,Ship To Address Number,Ship To Name Precision,Ship To Name Recall,Ship To Name F1,Ship To Name Number,Total Amount Precision,Total Amount Recall,Total Amount F1,Total Amount Number,Total Tax Amount Precision,Total Tax Amount Recall,Total Tax Amount F1,Total Tax Amount Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,3.157200,2.761560,0.000000,0.000000,0.000000,35,0.000000,0.000000,0.000000,24,0.000000,0.000000,0.000000,11,0.000000,0.000000,0.000000,25,0.000000,0.000000,0.000000,17,0.000000,0.000000,0.000000,26,0.000000,0.000000,0.000000,40,0.000000,0.000000,0.000000,34,0.000000,0.000000,0.000000,23,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,0.736924
2,2.615900,2.500236,0.000000,0.000000,0.000000,35,0.000000,0.000000,0.000000,24,0.000000,0.000000,0.000000,11,0.000000,0.000000,0.000000,25,0.000000,0.000000,0.000000,17,0.000000,0.000000,0.000000,26,0.000000,0.000000,0.000000,40,0.000000,0.000000,0.000000,34,0.000000,0.000000,0.000000,23,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,0.736924
3,2.520200,2.419673,0.000000,0.000000,0.000000,35,0.000000,0.000000,0.000000,24,0.000000,0.000000,0.000000,11,0.000000,0.000000,0.000000,25,0.000000,0.000000,0.000000,17,0.000000,0.000000,0.000000,26,0.000000,0.000000,0.000000,40,0.000000,0.000000,0.000000,34,0.000000,0.000000,0.000000,23,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,0.736924
4,2.400100,2.378855,0.000000,0.000000,0.000000,35,0.000000,0.000000,0.000000,24,0.000000,0.000000,0.000000,11,0.000000,0.000000,0.000000,25,0.000000,0.000000,0.000000,17,0.000000,0.000000,0.000000,26,0.000000,0.000000,0.000000,40,0.000000,0.000000,0.000000,34,0.000000,0.000000,0.000000,23,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,0.736924
5,2.372900,2.345494,0.000000,0.000000,0.000000,35,0.000000,0.000000,0.000000,24,0.000000,0.000000,0.000000,11,0.000000,0.000000,0.000000,25,0.000000,0.000000,0.000000,17,0.000000,0.000000,0.000000,26,0.000000,0.000000,0.000000,40,0.000000,0.000000,0.000000,34,0.000000,0.000000,0.000000,23,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,0.736924
6,2.404000,2.313804,0.000000,0.000000,0.000000,35,0.000000,0.000000,0.000000,24,0.000000,0.000000,0.000000,11,0.000000,0.000000,0.000000,25,0.000000,0.000000,0.000000,17,0.000000,0.000000,0.000000,26,0.000000,0.000000,0.000000,40,0.000000,0.000000,0.000000,34,0.000000,0.000000,0.000000,23,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,0.736924
7,2.331700,2.281004,0.000000,0.000000,0.000000,35,0.000000,0.000000,0.000000,24,0.000000,0.000000,0.000000,11,0.000000,0.000000,0.000000,25,0.000000,0.000000,0.000000,17,0.000000,0.000000,0.000000,26,0.000000,0.000000,0.000000,40,0.000000,0.000000,0.000000,34,0.000000,0.000000,0.000000,23,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,0.736924
8,2.274100,2.243464,0.000000,0.000000,0.000000,35,0.000000,0.000000,0.000000,24,0.000000,0.000000,0.000000,11,0.000000,0.000000,0.000000,25,0.000000,0.000000,0.000000,17,0.000000,0.000000,0.000000,26,0.000000,0.000000,0.000000,40,0.000000,0.000000,0.000000,34,0.000000,0.000000,0.000000,23,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,0.736924
9,2.243700,2.203809,0.000000,0.000000,0.000000,35,0.000000,0.000000,0.000000,24,0.000000,0.000000,0.000000,11,0.000000,0.000000,0.000000,25,0.000000,0.000000,0.000000,17,0.000000,0.000000,0.000000,26,0.000000,0.000000,0.000000,40,0.000000,0.

TrainOutput(global_step=140, training_loss=2.290476206370762, metrics={'train_runtime': 377.8414, 'train_samples_per_second': 1.323, 'train_steps_per_second': 0.371, 'total_flos': 269951588352000.0, 'train_loss': 2.290476206370762, 'epoch': 20.0})